# No Installation Required

microsoft/Phi-3-mini-4k-instruct + LoRA > GPU Parallel Training

The max sequence length has a significant impact on model performance, 
but due to insufficient memory, it was set to a maximum length of 768.


## Load Library

In [ ]:
import multiprocessing as mp
mp.set_start_method('spawn')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

import datasets
from datasets import load_dataset, load_metric, Dataset

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, log_loss

from accelerate import notebook_launcher, Accelerator, PartialState
from accelerate.utils import write_basic_config
from accelerate.inference import prepare_pippy

import transformers
from transformers import (
    AdamW,
    get_linear_schedule_with_warmup,
    set_seed,
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    AutoConfig
)

import os
import shutil
import math
import json
from tqdm import tqdm
import gc
import pandas as pd
import numpy as np
from typing import Optional,Tuple

In [ ]:
# params
model_name = "/kaggle/input/microsoftphi-3-mini-4k-instruct/transformers/default/1"
model_path = "model_checkpoint.pth"
seed = 42
lora_r = 2
quantize_bit = 16
learning_rate = 5e-4
weight_decay = 0.1
beta1 = 0.9
beta2 = 0.999
eps = 1e-9
l1_rate = 1e-10
batch_size = 1
max_len = 256
n_sample = 0.10
n_epoch = 2
device = "cuda"
file_path = '/kaggle/input/lmsys-chatbot-arena/train.csv'

## Preprocess

In [ ]:
def cl(x):
  if x == [1,0,0]:
    return 0
  elif x == [0,1,0]:
    return 1
  else :
    return 2

In [ ]:
def preprocess_data(file_path, sample = False):
    train = pd.read_csv(file_path)
    clf_train = train[['prompt','response_a','response_b','winner_model_a','winner_model_b','winner_tie']]

    clf_train.loc[:, "prompt"] = clf_train["prompt"].apply(lambda x: json.loads(x)[0])
    clf_train.loc[:, "response_a"] = clf_train["response_a"].apply(lambda x: json.loads(x)[0])
    clf_train.loc[:, "response_b"] = clf_train["response_b"].apply(lambda x: json.loads(x)[0])

    clf_train = clf_train.dropna()
    clf_train = clf_train.reset_index(drop = True)

    clf_train['target'] = [[clf_train['winner_model_a'][x],clf_train['winner_model_b'][x],clf_train['winner_tie'][x]] for x in range(len(clf_train)) ]

    clf_train = clf_train[['prompt','response_a','response_b','target']]

    clf_train['labels'] = clf_train['target'].apply(lambda x : cl(x))

    clf_train['p_len'] = clf_train['prompt'].apply(lambda x : len(x))
    clf_train['a_len'] = clf_train['response_a'].apply(lambda x : len(x))
    clf_train['b_len'] = clf_train['response_b'].apply(lambda x : len(x))

    clf_train['len'] = clf_train['p_len'] + clf_train['a_len']+ clf_train['b_len']
    
    if sample:
        clf_train = clf_train.sample(int(len(clf_train)*n_sample), weights = "len", random_state=seed).reset_index(drop=True)

    t_dat, v_dat = train_test_split(clf_train, test_size=0.2, random_state=42, stratify = clf_train['labels'])

    t_dat = t_dat.reset_index(drop=True)
    v_dat = v_dat.reset_index(drop=True)

    t_dat = t_dat.drop( labels= 'target' , axis = 1)
    v_dat = v_dat.drop( labels= 'target' , axis = 1)
    return t_dat, v_dat

In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.prompt = df['prompt']
        self.response_a = df['response_a']
        self.response_b = df['response_b']
        self.max_len = max_len
        self.targets = df.get('labels', None)

    def __len__(self):
        return len(self.prompt)

    def __getitem__(self, index):
        prompt = str(self.prompt[index])
        response_a = str(self.response_a[index])
        response_b = str(self.response_b[index])

        prompt_len = len(self.tokenizer("##prompt: " + prompt, add_special_tokens=True)['input_ids'])
        response_a_len = len(self.tokenizer("##response_a: " + response_a, add_special_tokens=True)['input_ids'])
        response_b_len = len(self.tokenizer("##response_b: " + response_b, add_special_tokens=True)['input_ids'])

        final_prompt_len = min(self.max_len, prompt_len)
        final_a_len = min(self.max_len, response_a_len)
        final_b_len = min(self.max_len, response_b_len)

        prompt_token = self.tokenizer("##prompt: " + prompt, add_special_tokens=True, max_length=final_prompt_len, truncation=True,padding='max_length', return_attention_mask=True, return_tensors='pt')
        response_a_token = self.tokenizer("##response_a: " + response_a, add_special_tokens=True, max_length=final_a_len, truncation=True,padding='max_length', return_attention_mask=True, return_tensors='pt')
        response_b_token = self.tokenizer("##response_b: " + response_b, add_special_tokens=True, max_length=final_b_len, truncation=True,padding='max_length', return_attention_mask=True, return_tensors='pt')

        input_ids = torch.cat([prompt_token['input_ids'], response_a_token['input_ids'], response_b_token['input_ids']], dim=1)
        attention_mask = torch.cat([prompt_token['attention_mask'], response_a_token['attention_mask'], response_b_token['attention_mask']], dim=1)

        if self.targets is not None:
            labels = torch.LongTensor([self.targets[index]])
            return {'input_ids': input_ids.flatten(), 'attention_mask': attention_mask.flatten(), 'labels': labels}
        else:
            return {'input_ids': input_ids.flatten(), 'attention_mask': attention_mask.flatten()}

In [ ]:
def custom_collate_fn(batch, tokenizer):

    input_ids = [item['input_ids'] for item in batch]
    attention_masks = [item['attention_mask'] for item in batch]
    labels = torch.cat([item['labels'] for item in batch], dim=0) if 'labels' in batch[0] else None

    # Find the maximum length of the sequences in the batch
    max_len = max([input_id.size(0) for input_id in input_ids])

    # Re-tokenize with the new max length
    new_input_ids = []
    new_attention_masks = []

    for item in batch:
        input_ids = item['input_ids'][:max_len]
        attention_mask = item['attention_mask'][:max_len]

        new_input_ids.append(input_ids)
        new_attention_masks.append(attention_mask)

    new_input_ids = pad_sequence(new_input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    new_attention_masks = pad_sequence(new_attention_masks, batch_first=True, padding_value=0)

    output = {
    'input_ids': new_input_ids,
    'attention_mask': new_attention_masks}

    if labels is not None:
        output['labels'] = labels

    return output

In [ ]:
def create_dataloaders(df,tokenizer,max_len, batch_size, shuffle = True):
    dataloader = DataLoader(
        CustomDataset(df, tokenizer, max_len), shuffle=shuffle, batch_size=batch_size , collate_fn=lambda x: custom_collate_fn(x, tokenizer)
    )
    return dataloader

## Load Model

In [ ]:
def quantize_tensor(tensor, num_bits=quantize_bit):
    qmin = 0.
    qmax = 2.**num_bits - 1.

    min_val, max_val = tensor.min(), tensor.max()
    scale = (max_val - min_val) / (qmax - qmin)
    zero_point = qmin - min_val / scale

    quantized_tensor = torch.round(tensor / scale + zero_point)
    quantized_tensor = torch.clamp(quantized_tensor, qmin, qmax)
    quantized_tensor = (quantized_tensor - zero_point) * scale

    return quantized_tensor

def quantize_model(model, num_bits=8):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            module.weight.data = quantize_tensor(module.weight.data, num_bits)
            if module.bias is not None:
                module.bias.data = quantize_tensor(module.bias.data, num_bits)
        elif isinstance(module, nn.Conv2d):
            module.weight.data = quantize_tensor(module.weight.data, num_bits)
            if module.bias is not None:
                module.bias.data = quantize_tensor(module.bias.data, num_bits)

    return model


# import torch.quantization

# def quantize_model_dynamic(model):
#     model.qconfig = torch.quantization.default_dynamic_qconfig
#     torch.quantization.prepare(model, inplace=True)
#     torch.quantization.convert(model, inplace=True)
#     return model

## Add Adepter Layers
Copied from transformers.models.phi3.modeling_phi3.Phi3Attention

[github url](https://github.com/huggingface/transformers/blob/main/src/transformers/models/phi3/modeling_phi3.py)

In [ ]:
class LoRA(nn.Module):
    def __init__(self, in_features, out_features, rank=lora_r, alpha=1.0, lora_dropout = 0.05):
        super(LoRA, self).__init__()
        self.alpha = alpha
        self.rank = rank
        self.lora_a = nn.Linear(in_features, rank, bias=False)
        self.lora_b = nn.Linear(rank, out_features, bias=False)
        self.dropout = nn.Dropout(lora_dropout)

    def forward(self, x):
        lora_out =  self.alpha * self.lora_b(self.lora_a(x))
        lora_out = self.dropout(lora_out)
        return lora_out

In [ ]:
from transformers.models.phi3.modeling_phi3 import (
Phi3RotaryEmbedding, 
# Phi3LongRoPEScaledRotaryEmbedding,
apply_rotary_pos_emb,
repeat_kv
)

In [ ]:
class Phi3Attention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config, layer_idx: Optional[int] = None):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx
        if layer_idx is None:
            logger.warning_once(
                f"Instantiating {self.__class__.__name__} without passing a `layer_idx` is not recommended and will "
                "lead to errors during the forward call if caching is used. Please make sure to provide a `layer_idx` "
                "when creating this class."
            )

        self.attention_dropout = config.attention_dropout
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.hidden_size // self.num_heads
        self.num_key_value_heads = config.num_key_value_heads
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads
        self.max_position_embeddings = config.max_position_embeddings
        self.original_max_position_embeddings = config.original_max_position_embeddings
        self.rope_theta = config.rope_theta
        self.rope_scaling = config.rope_scaling
        self.is_causal = True

        if (self.head_dim * self.num_heads) != self.hidden_size:
            raise ValueError(
                f"hidden_size must be divisible by num_heads (got `hidden_size`: {self.hidden_size}"
                f" and `num_heads`: {self.num_heads})."
            )

        op_size = self.num_heads * self.head_dim + 2 * (self.num_key_value_heads * self.head_dim)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=False)
        self.qkv_proj = nn.Linear(self.hidden_size, op_size, bias=False)
        self._init_rope()
        
        ########################## LoRA adapters ##########################
        self.qkv_lora = LoRA(self.hidden_size, op_size, lora_r)
        self.o_lora = LoRA(self.num_heads * self.head_dim, self.hidden_size, lora_r)
        ########################## LoRA adapters ##########################
        
    def _init_rope(self):
        if self.rope_scaling is None:
            self.rotary_emb = Phi3RotaryEmbedding(
                self.head_dim,
                max_position_embeddings=self.max_position_embeddings,
                base=self.rope_theta,
            )
        else:
            scaling_type = self.config.rope_scaling["type"]
            if scaling_type == "longrope":
                self.rotary_emb = Phi3LongRoPEScaledRotaryEmbedding(self.head_dim, self.config)
            else:
                raise ValueError(f"Unknown RoPE scaling type {scaling_type}")

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_value = None,
        output_attentions: bool = False,
        use_cache: bool = False,
        cache_position: Optional[torch.LongTensor] = None,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
#         logger.warning_once("You are not running the flash-attention implementation, expect numerical differences.")

        bsz, q_len, _ = hidden_states.size()
        ########################## LoRA adapters ##########################
        qkv = self.qkv_proj(hidden_states) + self.qkv_lora(hidden_states)
        ########################## LoRA adapters ##########################
        query_pos = self.num_heads * self.head_dim
        query_states = qkv[..., :query_pos]
        key_states = qkv[..., query_pos : query_pos + self.num_key_value_heads * self.head_dim]
        value_states = qkv[..., query_pos + self.num_key_value_heads * self.head_dim :]

        query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

        kv_seq_len = key_states.shape[-2]
        if past_key_value is not None:
            if self.layer_idx is None:
                raise ValueError(
                    f"The cache structure has changed since version v4.36. If you are using {self.__class__.__name__} "
                    "for auto-regressive decoding with k/v caching, please make sure to initialize the attention class "
                    "with a layer index."
                )
            kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
        cos, sin = self.rotary_emb(value_states, position_ids, seq_len=kv_seq_len)

        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

        if past_key_value is not None:
            cache_kwargs = {"sin": sin, "cos": cos, "cache_position": cache_position}  # Specific to RoPE models
            key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)

        # repeat k/v heads if n_kv_heads < n_heads
        key_states = repeat_kv(key_states, self.num_key_value_groups)
        value_states = repeat_kv(value_states, self.num_key_value_groups)

        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)

        if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):
            raise ValueError(
                f"Attention weights should be of size {(bsz, self.num_heads, q_len, kv_seq_len)}, but is"
                f" {attn_weights.size()}"
            )

        if attention_mask is not None:
            causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]
            attn_weights += causal_mask

        # upcast attention to fp32
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(value_states.dtype)
        attn_weights = nn.functional.dropout(attn_weights, p=self.attention_dropout, training=self.training)

        attn_output = torch.matmul(attn_weights, value_states)

        if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(bsz, self.num_heads, q_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )

        attn_output = attn_output.transpose(1, 2).contiguous()
        attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)
        ########################## LoRA adapters ##########################
        attn_output = self.o_proj(attn_output) + self.o_lora(attn_output)
        ########################## LoRA adapters ##########################
        if not output_attentions:
            attn_weights = None

        return attn_output, attn_weights, past_key_value


In [ ]:
def replace_attention_module(config,layer,layer_idx):
    if hasattr(layer, 'self_attn') and layer_idx > 12:

        new_attention = Phi3Attention(config,layer_idx)

        new_attention.qkv_proj.weight.data.copy_(layer.self_attn.qkv_proj.weight.data)
        new_attention.o_proj.weight.data.copy_(layer.self_attn.o_proj.weight.data)

        layer.self_attn = new_attention

In [ ]:
loss_fn = nn.CrossEntropyLoss()

class LoraModelForClassification(nn.Module):
    def __init__(self, lora_model):  # config 추가
        super(LoraModelForClassification, self).__init__()
        self.config = lora_model.config  # config 설정
        self.peft_model = lora_model
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.config.hidden_size, 3)
#         self.classifier.weight.data = self.classifier.weight.data.to(torch.float16)
#         self.classifier.bias.data = self.classifier.bias.data.to(torch.float16)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.peft_model(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state.mean(dim =1)
        output_dropout = self.dropout(pooled_output)
        logits = self.classifier(output_dropout)
        loss = None
        if labels is not None:
          labels = labels
          loss = loss_fn(logits, labels)
        return loss, logits

## Parallel Train

In [ ]:
def parallel_function(model_name,attention_name,file_path):
    mp.set_start_method('spawn', force=True)

    accelerator = Accelerator(mixed_precision="fp16")
    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    set_seed(seed)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training
    
    model = AutoModel.from_pretrained(model_name,torch_dtype=torch.float16)
    model = quantize_model(model)
    for idx, layer in enumerate(model.layers):
        replace_attention_module(model.config,layer,idx)
    model = LoraModelForClassification(model)

    for param in model.peft_model.parameters():
        param.requires_grad = False
    for param in model.classifier.parameters():
        param.requires_grad = True

    for name, module in model.named_modules():
        if isinstance(module, attention_name):
            module.qkv_lora.lora_a.weight.requires_grad = True
            module.qkv_lora.lora_b.weight.requires_grad = True
            module.o_lora.lora_a.weight.requires_grad = True
            module.o_lora.lora_b.weight.requires_grad = True

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad == True)
    print(f"Total trainable parameters: {total_params}")

    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    
    t_dat, v_dat = preprocess_data(file_path,sample = True)
    train_dataloader = create_dataloaders(t_dat,tokenizer,max_len,batch_size=batch_size, shuffle = True)
    eval_dataloader = create_dataloaders(v_dat,tokenizer,max_len,batch_size=batch_size, shuffle = True)

    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader)

    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=100,
        num_training_steps=len(train_dataloader) * n_epoch,
    )

    progress_bar = tqdm(range(n_epoch * len(train_dataloader)), disable=not accelerator.is_main_process)

    train_loss = 0
    valid_loss = 0
    
    for epoch in range(n_epoch):
        model.train()
        for step, batch in enumerate(train_dataloader):
            loss, _ = model(**batch)
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

            train_loss += loss.item()
        
        all_predictions = []
        all_labels = []
        model.eval()
        for step, batch in enumerate(eval_dataloader):
            with torch.no_grad():
                loss, logits = model(**batch)
            predictions = logits.argmax(dim=-1)
            all_predictions.append(accelerator.gather(predictions))
            all_labels.append(accelerator.gather(batch["labels"]))
            
            valid_loss += loss.item()

        all_predictions = torch.cat(all_predictions)[:len(eval_dataloader)].cpu()
        all_labels = torch.cat(all_labels)[:len(eval_dataloader)].cpu()

        acc_metric = accuracy_score(all_labels, all_predictions)
        eval_metric = f1_score(all_labels, all_predictions, average="macro")
        train_loss_avg = train_loss / len(train_dataloader)
        valid_loss_avg = valid_loss / len(eval_dataloader)
        
        accelerator.print(f"epoch: {epoch} \n accuracy: {acc_metric:.3f} \n f1 score: {eval_metric:.3f} \n train loss: {train_loss_avg:.3f} \n valid loss: {valid_loss_avg:.3f}")

    model = accelerator.unwrap_model(model)
    accelerator.wait_for_everyone()

    # 모델 상태 사전 저장
    if accelerator.is_main_process:
        torch.save(model.state_dict(), model_path)

    # 동기화 완료 메시지
    accelerator.wait_for_everyone()

In [ ]:
notebook_launcher(parallel_function, args=(model_name,Phi3Attention,file_path,), num_processes=2)